In [78]:
# Install Library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [79]:
# Ignore Warning
from warnings import filterwarnings
filterwarnings('ignore')


In [80]:
#Load DataSet
#read in the dataset (select 2011 to 2015)
# year11 = '2011'
# year12 = '2012'
# year13 = '2013'
# year14 = '2014'
# brfss_2011_dataset = pd.read_csv(f'C:\Github\Diabetes\Diabetes\Data\{year11}.csv')
# brfss_2012_dataset = pd.read_csv(f'C:\Github\Diabetes\Diabetes\Data\{year12}.csv')
# brfss_2013_dataset = pd.read_csv(f'C:\Github\Diabetes\Diabetes\Data\{year13}.csv')
# brfss_2014_dataset = pd.read_csv(f'C:\Github\Diabetes\Diabetes\Data\{year14}.csv')

year15 = '2015'
brfss_2015_dataset = pd.read_csv(f'C:\Github\Diabetes\Diabetes\Data\{year15}.csv')
brfss_2015_dataset.info

<bound method DataFrame.info of         _STATE  FMONTH        IDATE IMONTH   IDAY    IYEAR  DISPCODE  \
0          1.0     1.0  b'01292015'  b'01'  b'29'  b'2015'    1200.0   
1          1.0     1.0  b'01202015'  b'01'  b'20'  b'2015'    1100.0   
2          1.0     1.0  b'02012015'  b'02'  b'01'  b'2015'    1200.0   
3          1.0     1.0  b'01142015'  b'01'  b'14'  b'2015'    1100.0   
4          1.0     1.0  b'01142015'  b'01'  b'14'  b'2015'    1100.0   
...        ...     ...          ...    ...    ...      ...       ...   
441451    72.0    11.0  b'12162015'  b'12'  b'16'  b'2015'    1100.0   
441452    72.0    11.0  b'12142015'  b'12'  b'14'  b'2015'    1100.0   
441453    72.0    11.0  b'12232015'  b'12'  b'23'  b'2015'    1200.0   
441454    72.0    11.0  b'12152015'  b'12'  b'15'  b'2015'    1100.0   
441455    72.0    11.0  b'12152015'  b'12'  b'15'  b'2015'    1100.0   

               SEQNO          _PSU  CTELENUM  ...  _PAREC1  _PASTAE1  \
0       2.015000e+09  2.015000e

In [81]:

# select specific columns, based on BRFSS 2015 Codebook: https://www.cdc.gov/brfss/annual_data/2015/pdf/codebook15_llcp.pdf
brfss_df_selected = brfss_2015_dataset[['DIABETE3',
                                         '_RFHYPE5',  
                                         'TOLDHI2', '_CHOLCHK', 
                                         '_BMI5', 
                                         'SMOKE100', 
                                         'CVDSTRK3', '_MICHD', 
                                         '_TOTINDA', 
                                         '_FRTLT1', '_VEGLT1', 
                                         '_RFDRHV5', 
                                         'HLTHPLN1', 'MEDCOST', 
                                         'GENHLTH', 'MENTHLTH', 'PHYSHLTH', 'DIFFWALK', 
                                         'SEX', '_AGEG5YR', 'EDUCA', 'INCOME2' ]]

brfss_df_selected.info


<bound method DataFrame.info of         DIABETE3  _RFHYPE5  TOLDHI2  _CHOLCHK   _BMI5  SMOKE100  CVDSTRK3  \
0            3.0       2.0      1.0       1.0  4018.0       1.0       2.0   
1            3.0       1.0      2.0       2.0  2509.0       1.0       2.0   
2            3.0       1.0      1.0       1.0  2204.0       NaN       1.0   
3            3.0       2.0      1.0       1.0  2819.0       2.0       2.0   
4            3.0       1.0      2.0       1.0  2437.0       2.0       2.0   
...          ...       ...      ...       ...     ...       ...       ...   
441451       1.0       2.0      1.0       1.0  1842.0       2.0       2.0   
441452       3.0       1.0      2.0       1.0  2834.0       2.0       2.0   
441453       3.0       2.0      1.0       1.0  4110.0       1.0       2.0   
441454       3.0       2.0      2.0       1.0  2315.0       2.0       2.0   
441455       1.0       2.0      1.0       1.0  2469.0       2.0       2.0   

        _MICHD  _TOTINDA  _FRTLT1  ...  HLT

##Modify and Clean Data

In [82]:
#Step 1 - Remove Missing Values records
brfss_df_selected = brfss_df_selected.dropna()
brfss_df_selected.shape

(343606, 22)

#Step 2 -  Modify and clean the values to be more suitable to ML algorithms
In order to do this part, I referenced the codebook which says what each column/feature/question is
BRFSS 2015 Codebook: https://www.cdc.gov/brfss/annual_data/2015/pdf/codebook15_llcp.pdf

In [83]:
# 1 DIABETE3 (Column 111, data value provied as below)
# Description: (Ever told) you have diabetes  (If "Yes" and respondent is female, ask "Was this only when you were pregnant?". 
# If Respondent says pre-diabetes or borderline diabetes, use response code 4.) 

# 1 = Yes
# 2 = Yes, But female told only during pregnancy
# 3 = No
# 4 = No, pre-diabetes or borderline diabetes
# 7 = Don't know or not sure
# 9 = Refused

# Modify data value
# 1 = changed to 2 
# 2 = changed to 0 
# 3 = changed to 0
# 4 = changed to 1
# 7 = removed records
# 9 = removed records

print("before", brfss_df_selected.DIABETE3.unique())
brfss_df_selected['DIABETE3'] = brfss_df_selected['DIABETE3'].replace({2:0, 3:0, 1:2, 4:1})
brfss_df_selected = brfss_df_selected[brfss_df_selected.DIABETE3 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.DIABETE3 != 9]
print("After", brfss_df_selected.DIABETE3.unique())
brfss_df_selected.shape

before [3. 1. 4. 2. 7. 9.]
After [0. 2. 1.]


(343232, 22)

In [84]:
# 2 _RFHYPE5 (Column 1896, data value provided as below)
#Description: Adults who have been told they have high blood pressure by a doctor, nurse, or other health professional

# 1 = No
# 2 = Yes
# 9 = Don’t know/Not Sure/Refused/Missing

# Modify data value 
# 1 = changed to 0
# 2 = Changed to 1 
# 9 = Removed records
#Change 1 to 0 so it represetnts No high blood pressure and 2 to 1 so it represents high blood pressure

print("Before", brfss_df_selected._RFHYPE5.unique())
brfss_df_selected['_RFHYPE5'] = brfss_df_selected['_RFHYPE5'].replace({1:0, 2:1})
brfss_df_selected = brfss_df_selected[brfss_df_selected._RFHYPE5 != 9]
print("After", brfss_df_selected._RFHYPE5.unique())
brfss_df_selected.shape

Before [2. 1. 9.]
After [1. 0.]


(342539, 22)

In [85]:
# 3 TOLDHI2 (Column 1898, data value provided as below)
# Description: Adults who have had their cholesterol checked and have been told by a doctor, nurse, 
# or other health professional that it was high

# 1 = No
# 2 = Yes
# 9 = Don’t know/Not Sure/Refused/Missing

# Modify data value 
# 1 = Unchange
# 2 = change to 0
# 7 = Removed records
# 9 = Removed records


print("Before", brfss_df_selected.TOLDHI2.unique())
brfss_df_selected['TOLDHI2'] = brfss_df_selected['TOLDHI2'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.TOLDHI2 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.TOLDHI2 != 9]
print("After", brfss_df_selected.TOLDHI2.unique())
brfss_df_selected.shape

Before [1. 2. 7. 9.]
After [1. 0.]


(339832, 22)

In [86]:
# 4 _CHOLCHK (Column 1897, data value provided as below)
# Description: Cholesterol check within past five years 
 
# 1 = Had cholesterol checked in past 5 years
# 2 = Did not have cholesterol checked in past 5 years
# 3 = Have never had cholesterol checked 
# 9 = Don’t know/Not Sure/Refused/Missing

# Modify data value 
# 1 = Unchange
# 2 = change to 0
# 3 = change to 0
# 9 = Removed records

print("Before", brfss_df_selected._CHOLCHK.unique())
brfss_df_selected['_CHOLCHK'] = brfss_df_selected['_CHOLCHK'].replace({3:0,2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected._CHOLCHK != 9]
print("After", brfss_df_selected._CHOLCHK.unique())
brfss_df_selected.shape

Before [1. 2. 9.]
After [1. 0.]


(335490, 22)

In [87]:
# 5 _BMI5 (Column 1898 ~ 1991, data value provided as below)
# Description: Body Mass Index (BMI) 

# 1 - 9999 1 or greater

# Modify data value
# BMI * 100, to remove decimal points for example a BMI of 4018 is really 40.18

print("Before", brfss_df_selected._BMI5.unique())
brfss_df_selected['_BMI5'] = brfss_df_selected['_BMI5'].div(100).round(0)
brfss_df_selected._BMI5.unique()
print("After", brfss_df_selected._BMI5.unique())
brfss_df_selected.shape

Before [4018. 2509. 2819. ... 4159. 1444. 6076.]
After [40. 25. 28. 24. 27. 30. 26. 23. 34. 33. 21. 22. 31. 38. 20. 19. 32. 46.
 41. 37. 36. 29. 35. 18. 54. 45. 39. 47. 43. 55. 49. 42. 17. 16. 48. 44.
 50. 59. 15. 52. 53. 57. 51. 14. 58. 63. 61. 56. 60. 74. 62. 64. 13. 66.
 73. 65. 68. 85. 71. 84. 67. 70. 82. 79. 92. 72. 88. 96. 81. 12. 77. 95.
 75. 91. 69. 76. 87. 89. 83. 98. 86. 80. 90. 78. 97.]


(335490, 22)

In [88]:
# 6 SMOKE100 (Column 194, data value provided as below)
# Description: Have you smoked at least 100 cigarettes in your entire life?   [Note:  5 packs = 100 cigarettes]

# 1 = Yes
# 2 = No
# 7 = Don’t know/Not Sure
# 9 = Refused

# Modify data value
# 1 = Unchange
# 2 = change to 0
# 7 = Removed records
# 9 = Removed records

print("Before", brfss_df_selected.SMOKE100.unique())
brfss_df_selected['SMOKE100'] = brfss_df_selected['SMOKE100'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.SMOKE100 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.SMOKE100 != 9]
print("After", brfss_df_selected.SMOKE100.unique())
brfss_df_selected.shape

Before [1. 2. 7. 9.]
After [1. 0.]


(333529, 22)

In [89]:
# 7 CVDSTRK3 (Column 108, data value provided as below)
# Description: (Ever told) you had a stroke

# 1 = Yes
# 2 = No
# 7 = Don’t know/Not Sure
# 9 = Refused

# Modify data value
# 1 = Unchange
# 2 = change to 0
# 7 = Removed records
# 9 = Removed records

print("Before", brfss_df_selected.CVDSTRK3.unique())
brfss_df_selected['CVDSTRK3'] = brfss_df_selected['CVDSTRK3'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.CVDSTRK3 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.CVDSTRK3 != 9]
print("After", brfss_df_selected.CVDSTRK3.unique())
brfss_df_selected.shape

Before [2. 1. 7. 9.]
After [0. 1.]


(332801, 22)

In [90]:
# 8 _MICHD (Column 1899, data value provided as below)
# Description: Respondents that have ever reported having coronary heart disease (CHD) or myocardial infarction (MI) 

# 1 = Reported having MI or CHD 
# 2 = Did not report having MI or CHD 


# Modify data value
# 1 = Unchange
# 2 = change to 0


print("Before", brfss_df_selected._MICHD.unique())
brfss_df_selected['_MICHD'] = brfss_df_selected['_MICHD'].replace({2: 0})
brfss_df_selected._MICHD.unique()
print("After", brfss_df_selected._MICHD.unique())
brfss_df_selected.shape

Before [2. 1.]
After [0. 1.]


(332801, 22)

In [91]:
# 9 _TOTINDA (Column 2058, data value provided as below)
# Description: Adults who reported doing physical activity or exercise during the past 30 days other than their regular job 

# 1 = Had physical activity or exercise
# 2 = No physical activity or exercise in last 30 days
# 9 = Don’t know/Refused/Missing

# Modify data value
# 1 = Unchange
# 2 = change to 0
# 9 = Removed records

print("Before", brfss_df_selected._TOTINDA.unique())
brfss_df_selected['_TOTINDA'] = brfss_df_selected['_TOTINDA'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected._TOTINDA != 9]
print("After", brfss_df_selected._TOTINDA.unique())
brfss_df_selected.shape

Before [2. 1. 9.]
After [0. 1.]


(317404, 22)

In [92]:
# 10 _FRTLT1 (Column 2050, data value provided as below)
# Description: Consume Fruit 1 or more times per day 

# 1 = Consumed fruit one or more times per day
# 2 = Consumed fruit less than one time per day 
# 9 = Don’t know/Refused/Missing

# Modify data value
# 1 = Unchange
# 2 = change to 0
# 9 = Removed records

print("Before", brfss_df_selected._FRTLT1.unique())
brfss_df_selected['_FRTLT1'] = brfss_df_selected['_FRTLT1'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected._FRTLT1 != 9]
print("After", brfss_df_selected._FRTLT1.unique())
brfss_df_selected.shape

Before [2. 1. 9.]
After [0. 1.]


(309768, 22)

In [93]:
# 11 _VEGLT1 (Column 2051, data value provided as below)
# Description: Consume Vegetables 1 or more times per day 

# 1 = Consumed vegetables one or more times per day 
# 2 = Consumed vegetables less than one time per day 
# 9 = Don’t know/Refused/Missing

# Modify data value
# 1 = Unchange
# 2 = change to 0
# 9 = Removed records

print("Before", brfss_df_selected._VEGLT1.unique())
brfss_df_selected['_VEGLT1'] = brfss_df_selected['_VEGLT1'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected._VEGLT1 != 9]
print("After", brfss_df_selected._VEGLT1.unique())
brfss_df_selected.shape

Before [1. 2. 9.]
After [1. 0.]


(302160, 22)

In [94]:
# 12 _RFDRHV5 (Column 2009, data value provided as below)
# Description: Heavy drinkers (adult men having more than 14 drinks per week and adult women having more than 7 drinks per week)

# 1 = No 
# 2 = Yes
# 9 = Don’t know/Refused/Missing

# Modify data value
# 1 = change to 0
# 2 = change to 1
# 9 = Removed records

print("Before", brfss_df_selected._RFDRHV5.unique())
brfss_df_selected['_RFDRHV5'] = brfss_df_selected['_RFDRHV5'].replace({1:0, 2:1})
brfss_df_selected = brfss_df_selected[brfss_df_selected._RFDRHV5 != 9]
print("After", brfss_df_selected._RFDRHV5.unique())
brfss_df_selected.shape

Before [1. 9. 2.]
After [0. 1.]


(298637, 22)

In [95]:
# 13 HLTHPLN1 (Column 97, data value provided as below)
# Description: Do you have any kind of health care coverage, including health insurance, prepaid plans such as HMOs, or government plans such as Medicare, or Indian Health Service? 

# 1 = Yes
# 2 = No
# 7 = Don’t know/Not Sure 
# 9 = Refused

# Modify data value
# 1 = Unchange
# 2 = change to 0
# 7 = Removed records
# 9 = Removed records

print("Before", brfss_df_selected.HLTHPLN1.unique())
brfss_df_selected['HLTHPLN1'] = brfss_df_selected['HLTHPLN1'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.HLTHPLN1 != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.HLTHPLN1 != 9]
print("After", brfss_df_selected.HLTHPLN1.unique())
brfss_df_selected.shape

Before [1. 2. 9. 7.]
After [1. 0.]


(298145, 22)

In [96]:
# 14 MEDCOST (Column 99, data value provided as below)
# Description: Was there a time in the past 12 months when you needed to see a doctor but could not because of cost?

# 1 = Yes
# 2 = No
# 7 = Don’t know/Not Sure 
# 9 = Refused

# Modify data value
# 1 = Unchange
# 2 = change to 0
# 7 = Removed records
# 9 = Removed records

print("Before", brfss_df_selected.MEDCOST.unique())
brfss_df_selected['MEDCOST'] = brfss_df_selected['MEDCOST'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.MEDCOST != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.MEDCOST != 9]
print("After", brfss_df_selected.MEDCOST.unique())
brfss_df_selected.shape

Before [2. 1. 7. 9.]
After [0. 1.]


(297709, 22)

In [97]:
# 15 GENHLTH (Column 90, data value provided as below)
# Description: Would you say that in general your health is: 

# 1 = Excellent
# 2 = Very good
# 3 = Good
# 4 = Fair
# 5 = Poor
# 7 = Don’t know/Not Sure
# 9 = Refused

# Modify data value
# 1 = Unchange
# 2 = Unchange
# 3 = Unchange
# 4 = Unchange
# 5 = Unchange
# 7 = Removed records
# 9 = Removed records
print("Before", brfss_df_selected.GENHLTH.unique())
brfss_df_selected = brfss_df_selected[brfss_df_selected.GENHLTH != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.GENHLTH != 9]
print("After", brfss_df_selected.GENHLTH.unique())
brfss_df_selected.shape

Before [5. 3. 2. 4. 1. 7. 9.]
After [5. 3. 2. 4. 1.]


(297194, 22)

In [98]:
# 16 MENTHLTH (Column 93-94, data value provided as below)
# Description: Now thinking about your mental health, which includes stress, depression, and problems with emotions, for how many 
# days during the past 30 days was your mental health not good?

# 1 ~ 30 = Number of days 
# 88 = None
# 77 = Don’t know/Not Sure
# 99 = Refused

# Modify data value
# 1 ~ 30 = Unchange
# 88 = change to 0
# 77 = Removed records
# 99 = Removed records

print("Before", brfss_df_selected.MENTHLTH.unique())
brfss_df_selected['MENTHLTH'] = brfss_df_selected['MENTHLTH'].replace({88:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.MENTHLTH != 77]
brfss_df_selected = brfss_df_selected[brfss_df_selected.MENTHLTH != 99]
print("After", brfss_df_selected.MENTHLTH.unique())
brfss_df_selected.shape

Before [18. 88. 30.  3. 99.  5. 15. 10.  6. 20.  2. 25.  1. 29.  4. 77.  7.  8.
 21. 14. 26.  9. 16. 28. 11. 12. 24. 17. 13. 23. 27. 19. 22.]
After [18.  0. 30.  3.  5. 15. 10.  6. 20.  2. 25.  1. 29.  4.  7.  8. 21. 14.
 26.  9. 16. 28. 11. 12. 24. 17. 13. 23. 27. 19. 22.]


(294102, 22)

In [99]:
# 17 PHYSHLTH (Column 91-92, data value provided as below)
# Description: Now thinking about your physical health, which includes physical illness and injury, for how many days during the past 
# 30 days was your physical health not good? 

# 1 ~ 30 = Number of days 
# 88 = None
# 77 = Don’t know/Not Sure
# 99 = Refused

# Modify data value
# 1 ~ 30 = Unchange
# 88 = change to 0
# 77 = Removed records
# 99 = Removed records

print("Before", brfss_df_selected.PHYSHLTH.unique())
brfss_df_selected['PHYSHLTH'] = brfss_df_selected['PHYSHLTH'].replace({88:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.PHYSHLTH != 77]
brfss_df_selected = brfss_df_selected[brfss_df_selected.PHYSHLTH != 99]
print("After", brfss_df_selected.MENTHLTH.unique())
brfss_df_selected.shape

Before [15. 88. 30.  2. 14. 28.  7. 77. 20.  3. 10.  1.  5. 17.  4. 19.  6. 21.
 12.  8. 25. 27. 22. 29. 24.  9. 16. 99. 18. 23. 13. 26. 11.]
After [18.  0. 30.  3.  5. 15. 10.  6. 20.  2. 25.  1.  4.  7.  8. 21. 14. 26.
  9. 29. 16. 28. 11. 12. 24. 17. 13. 23. 27. 19. 22.]


(290473, 22)

In [100]:
# 18 DIFFWALK (Column 191, data value provided as below)
# Description: Do you have serious difficulty walking or climbing stairs? 

# 1 = Yes
# 2 = No
# 7 = Don’t know/Not Sure 
# 9 = Refused

# Modify data value
# 1 = Unchange
# 2 = change to 0
# 7 = Removed records
# 9 = Removed records

print("Before", brfss_df_selected.DIFFWALK.unique())
brfss_df_selected['DIFFWALK'] = brfss_df_selected['DIFFWALK'].replace({2:0})
brfss_df_selected = brfss_df_selected[brfss_df_selected.DIFFWALK != 7]
brfss_df_selected = brfss_df_selected[brfss_df_selected.DIFFWALK != 9]
print("After", brfss_df_selected.DIFFWALK.unique())
brfss_df_selected.shape

Before [1. 2. 7. 9.]
After [1. 0.]


(289698, 22)

In [101]:
# 19 SEX (Column 120, data value provided as below)
# Description: Indicate sex of respondent. 

# 1 = Male
# 2 = Female

# Modify data value
# 1 = Unchange
# 2 = change to 0

print("Before", brfss_df_selected.SEX.unique())
brfss_df_selected['SEX'] = brfss_df_selected['SEX'].replace({2:0})
brfss_df_selected.SEX.unique()
print("After", brfss_df_selected.SEX.unique())
brfss_df_selected.shape

Before [2. 1.]
After [0. 1.]


(289698, 22)

In [102]:
# 20 _AGEG5YR (Column 1971 ~ 1972 , data value provided as below)
# Description: Fourteen-level age category 

# 1 = Age 18 to 24
# 2 = Age 25 to 29
# 3 = Age 30 to 34
# 4 = Age 35 to 39
# 5 = Age 40 to 44
# 6 = Age 45 to 49
# 7 = Age 50 to 54 
# 8 = Age 55 to 59
# 9 = Age 60 to 64
# 10 = Age 65 to 69
# 11 = Age 70 to 74
# 12 = Age 75 to 79 
# 13 = Age 80 or 99 (older) 
# 14 = Don’t know/Refused/Missing

# Modify data value
# 1 to 13 = Unchange
# 14 = Removed records

print("Before", brfss_df_selected._AGEG5YR.unique())
brfss_df_selected = brfss_df_selected[brfss_df_selected._AGEG5YR != 14]
print("After", brfss_df_selected._AGEG5YR.unique())
brfss_df_selected.shape

Before [ 9.  7. 11. 10. 13.  8.  4.  6.  2. 12.  5.  1.  3. 14.]
After [ 9.  7. 11. 10. 13.  8.  4.  6.  2. 12.  5.  1.  3.]


(288259, 22)

In [103]:
# 21 EDUCA (Column 158 , data value provided as below)
# Description: What is the highest grade or year of school you completed? 

# 1 = Never attended school or only kindergarten 
# 2 = Grades 1 through 8 (Elementary) 
# 3 = Grades 9 through 11 (Some high school)
# 4 = Grade 12 or GED (High school graduate)
# 5 = College 1 year to 3 years (Some college or technical school
# 6 = College 4 years or more (College graduate) 
# 9 = Refused

# Modify data value
# 1 to 6 = Unchange
# 9 = Removed records

print("Before", brfss_df_selected.EDUCA.unique())
brfss_df_selected = brfss_df_selected[brfss_df_selected.EDUCA != 9]
print("After", brfss_df_selected.EDUCA.unique())
brfss_df_selected.shape

Before [4. 6. 3. 5. 2. 1. 9.]
After [4. 6. 3. 5. 2. 1.]


(287931, 22)

In [104]:
# 22 INCOME2 (Column 175-176 , data value provided as below)
# Description: Is your annual household income from all sources:  (If respondent refuses at any income level, code "Refused.")

# 1 = Less than $10,000 
# 2 = Less than $15,000 ($10,000 to less than $15,000)  
# 3 = Less than $20,000 ($15,000 to less than $20,000)
# 4 = Less than $25,000 ($20,000 to less than $25,000) 
# 5 = Less than $35,000 ($25,000 to less than $35,000)
# 6 = Less than $50,000 ($35,000 to less than $50,000) 
# 7 = Less than $75,000 ($50,000 to less than $75,000)
# 8 = $75,000 or more 
# 77 = Don’t know/Not sure 
# 99 = Refused

# Modify data value
# 1 to 8 = Unchange
# 77 = Removed records
# 99 = Removed records

print("Before", brfss_df_selected.INCOME2.unique())
brfss_df_selected = brfss_df_selected[brfss_df_selected.INCOME2 != 77]
brfss_df_selected = brfss_df_selected[brfss_df_selected.INCOME2 != 99]
print("After", brfss_df_selected.INCOME2.unique())
brfss_df_selected.shape

Before [ 3.  1.  8.  6.  4.  7. 77.  2.  5. 99.]
After [3. 1. 8. 6. 4. 7. 2. 5.]


(253680, 22)

In [105]:
#Check the shape of the dataset now
brfss_df_selected.shape

(253680, 22)

In [106]:
#Let's see what the data looks like after Modifying Values
brfss_df_selected.head()

,DIABETE3,_RFHYPE5,TOLDHI2,_CHOLCHK,_BMI5,SMOKE100,CVDSTRK3,_MICHD,_TOTINDA,_FRTLT1,...,HLTHPLN1,MEDCOST,GENHLTH,MENTHLTH,PHYSHLTH,DIFFWALK,SEX,_AGEG5YR,EDUCA,INCOME2
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
3,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
5,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
6,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [107]:
#Check Class Sizes of the Diabete3 column
brfss_df_selected.groupby(['DIABETE3']).size()

DIABETE3
0.0    213703
1.0      4631
2.0     35346
dtype: int64

#Step 3 - Make feature names more readable

In [108]:
#Rename the columns to make them more readable
brfss = brfss_df_selected.rename(columns = {'DIABETE3':'Diabetes_012', 
                                         '_RFHYPE5':'HighBP',  
                                         'TOLDHI2':'HighChol', '_CHOLCHK':'CholCheck', 
                                         '_BMI5':'BMI', 
                                         'SMOKE100':'Smoker', 
                                         'CVDSTRK3':'Stroke', '_MICHD':'HeartDiseaseorAttack', 
                                         '_TOTINDA':'PhysActivity', 
                                         '_FRTLT1':'Fruits', '_VEGLT1':"Veggies", 
                                         '_RFDRHV5':'HvyAlcoholConsump', 
                                         'HLTHPLN1':'AnyHealthcare', 'MEDCOST':'NoDocbcCost', 
                                         'GENHLTH':'GenHlth', 'MENTHLTH':'MentHlth', 'PHYSHLTH':'PhysHlth', 'DIFFWALK':'DiffWalk', 
                                         'SEX':'Sex', '_AGEG5YR':'Age', 'EDUCA':'Education', 'INCOME2':'Income' })


In [109]:
brfss.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
3,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
5,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
6,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [110]:
#Check how many respondents have no diabetes, prediabetes or diabetes. Note the class imbalance!
brfss.groupby(['Diabetes_012']).size()

Diabetes_012
0.0    213703
1.0      4631
2.0     35346
dtype: int64

#Step 3 -Save to csv
First save version where diabetes is the target variable and in the first column. This is the full cleaned dataset with prediabetes still there.

In [111]:
#************************************************************************************************
brfss.to_csv('diabetes_012_health_indicators_BRFSS2015.csv', sep=",", index=False)
#************************************************************************************************

In [112]:
#Copy old table to new one.
brfss_binary = brfss
#Change the diabetics 2 to a 1 and pre-diabetics 1 to a 0, so that we have 0 meaning non-diabetic and pre-diabetic and 1 meaning diabetic.
brfss_binary['Diabetes_012'] = brfss_binary['Diabetes_012'].replace({1:0})
brfss_binary['Diabetes_012'] = brfss_binary['Diabetes_012'].replace({2:1})

#Change the column name to Diabetes_binary
brfss_binary = brfss_binary.rename(columns = {'Diabetes_012': 'Diabetes_binary'})
brfss_binary.Diabetes_binary.unique()

array([0., 1.])

In [113]:
#show class sizes
brfss_binary.groupby(['Diabetes_binary']).size()

Diabetes_binary
0.0    218334
1.0     35346
dtype: int64

In [117]:
#Separate the 0(No Diabetes) and 1&2(Pre-diabetes and Diabetes)
#Get the 1s
is1 = brfss_binary['Diabetes_binary'] == 1
brfss_5050_1 = brfss_binary[is1]

#Get the 0s
is0 = brfss_binary['Diabetes_binary'] == 0
brfss_5050_0 = brfss_binary[is0] 

#Select the 39977 random cases from the 0 (non-diabetes group). we already have 35346 cases from the diabetes risk group
brfss_5050_0_rand1 = brfss_5050_0.take(np.random.permutation(len(brfss_5050_0))[:35346])

#Append the 39977 1s to the 39977 randomly selected 0s


brfss_5050 = pd.concat([brfss_5050_0_rand1, brfss_5050_1], ignore_index=True)

In [118]:
print(f'brfss_5050={brfss_5050.shape}',f'brfss_binary={brfss_binary.shape}')

brfss_5050=(70692, 22) brfss_binary=(253680, 22)


In [119]:
#Save the 50-50 balanced dataset to csv
#************************************************************************************************
brfss_5050.to_csv('diabetes_binary_5050split_health_indicators_BRFSS2015.csv', sep=",", index=False)
#************************************************************************************************

#Also save the original binary dataset to csv
#************************************************************************************************
brfss_binary.to_csv('diabetes_binary_health_indicators_BRFSS2015.csv', sep=",", index=False)
#************************************************************************************************